<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/redisxann_usearch_implement_cross_modal_image_text_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run RedisXANN-usearch redis server

In [ ]:
!rm -rf redis && git clone https://github.com/redis/redis && \
  cd redis && \
  git checkout 7.0 && \
  make REDIS_CFLAGS='-Werror' BUILD_TLS=yes && make install && \
  which redis-server && redis-server --version

In [ ]:
!git clone https://github.com/weedge/RedisXANN.git --recursive

In [3]:
!curl https://sh.rustup.rs -sSf > rustup.sh

In [ ]:
!sh rustup.sh -y

In [ ]:
!cd RedisXANN && source "$HOME/.cargo/env" && \
  cargo clean && \
  cargo build --lib --manifest-path rust/usearch/Cargo.toml --release

In [6]:
!redis-server --daemonize yes \
  --loadmodule RedisXANN/target/release/libredisxann_usearch.so is_remove_serialized_file 1 \
  --port 6666 \
  --dbfilename dump.6666.rdb

## Data

1. Oxford-IIIT 宠物数据集 https://www.robots.ox.ac.uk/~vgg/data/pets/

 images.tar.gz（数据集）和 annotations.tar.gz（groundtruth 数据）

In [ ]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz -O images.tar.gz \
  && rm -rf images && tar -zxvf images.tar.gz

In [ ]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz -O annotations.tar.gz \
  && rm -rf annotations && tar -zxvf annotations.tar.gz

## install deps python package
torch、matplotlib、Pillow(colab defualt install)；安装redisx、CLIP(openai clip,阿里巴巴cn_clip)等依赖

In [ ]:
!pip install torch matplotlib Pillow clip cn_clip redisx

In [11]:
import os
from typing import List

from redis import ResponseError
import torch
from PIL import Image
import pylab
from matplotlib import pyplot as plt
import cn_clip.clip as clip
from cn_clip.clip import available_models


In [10]:
from random import random
from redisx.client import Client

# change the following configuration for your redis.
REDIS_HOST = "localhost"
REDIS_PORT = 6666
REDIS_DB = 0
REDIS_USERNAME = ""
REDIS_PASSWORD = ""


def get_client() -> Client:
    return Client(
        host=REDIS_HOST,
        port=REDIS_PORT,
        db=REDIS_DB,
        username=REDIS_USERNAME,
        password=REDIS_PASSWORD,
    )


def get_random_vectors(dim: int, n: int):
    return [[random() for _ in range(dim)] for _ in range(n)]

In [24]:
from typing import Union

from redis import ResponseError
from redisx.define import UsearchQuantizationType
from redisx.ann_usearch import VectorType

cli = get_client()

def create_index(index_name: str, dim:int):
    try:
        return cli.create_index(
            index_name, dim,
            quantization=UsearchQuantizationType.F64)
    except ResponseError as e:
        print(e)
        return None

def get_index(index_name: str):
    try:
        return cli.get_index(index_name)
    except ResponseError as e:
        print(e)
        return None

def add_vector(index_name: str, name: str, vector: Union[VectorType, str]):
    try:
        return cli.add_vector(index_name, name, vector)
    except ResponseError as e:
        print(e)
        return None

def kann_search(index_name: str, k: int, query_vector: Union[VectorType, str]):
    try:
        return cli.kann_search(index_name, k, query_vector)
    except ResponseError as e:
        print(e)
        return False

def build_index():
    """
    创建存储图片、文本向量的Vector索引：(使用usearch库建立索引)
    * 图片Key名称为"index_images"、文本Key名称为"index_texts"。
    * 向量维度为1024。
    * 计算向量距离函数为IP。
    """
    ret = get_index("index_images")
    if ret is None:
        create_index("index_images", 1024)
    ret = get_index("index_texts")
    if ret is None:
        create_index("index_texts", 1024)



In [25]:
build_index()


## load vision model
use ResNet-50

In [13]:
model, preprocess = clip.load_from_name("RN50", device="cuda", download_root="./")


100%|███████████████████████████████████████| 294M/294M [00:18<00:00, 16.9MiB/s]


Loading vision model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/RN50.json
Loading text model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}


In [ ]:
model.eval()

In [26]:
def insert_images(image_dir):
    """
    需要输入图片的路径，该方法会自动遍历路径下的图片文件。
    同时，该方法会调用extract_image_features方法（通过CLIP模型对图片文件进行预处理，并返回图片的特征信息），
    并行将返回的特征信息存入redisXAnn-usearch中。
    * 向量索引名称为“index_images”（固定）。
    * Key为图片路径及其文件名，例如“test/images/boxer_18.jpg”。
    * 特征信息为1024维向量。
    """
    file_names = [f for f in os.listdir(image_dir) if (f.endswith('.jpg') or f.endswith('.jpeg'))]
    for file_name in file_names:
        image_feature = extract_image_features(image_dir + "/" + file_name)
        add_vector("index_images", image_dir + "/" + file_name, image_feature)

def extract_image_features(img_name):
    """
    该方法将通过CLIP模型对图片文件进行预处理，并返回图片的特征信息（1024维向量）。
    """
    image_data = Image.open(img_name).convert("RGB")
    infer_data = preprocess(image_data)
    infer_data = infer_data.unsqueeze(0).to("cuda")
    with torch.no_grad():
        image_features = model.encode_image(infer_data)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    return image_features.cpu().numpy()[0]  # [1, 1024]



In [27]:
def insert_text(text):
    """
    需要输入需存储的文本，该方法会调用extract_text_features方法（通过CLIP模型对文本进行预处理，并返回文本的特征信息），
    并行将返回的特征信息存入redisXAnn-usearch中。
    存入Tair的格式为：
    * 向量索引名称为“index_texts”（固定）。
    * Key为文本内容，例如“奔跑的狗”。
    * 特征信息为1024维向量。
    """
    text_features = extract_text_features(text)
    add_vector("index_texts", text, text_features)


def extract_text_features(text):
    """
    该方法将通过CLIP模型对文本进行预处理，并返回文本的特征信息（1024维向量）。
    """
    text_data = clip.tokenize([text]).to("cuda")
    with torch.no_grad():
        text_features = model.encode_text(text_data)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features.cpu().numpy()[0]  # [1, 1024]



In [35]:
def query_images_by_text(q_text, topK):
    """
    该方法将用于以文搜图。
    需要输入待搜索的文本内容（text）和返回的结果数量（topK）。
    该方法会通过CLIP 模型将待查询的文本内容进行预处理，然后通过USEARCH.SEARCH.KANN命令，
    查询redis数据库中与该文本描述最为相似的图片。
    将返回目标图片名称name，id和相似值similarity，
    这里选择默认metric: IP(内积), 相似值similarity为距离，越小，表示相似度越高。
    """
    q_text_feature = extract_text_features(q_text)
    result = kann_search("index_images", topK, q_text_feature)
    print("topK {} q {} result {}",topK,q_text_feature,result)
    if result == None:
      return None
    for k, s in result["vals"]:
        print("{}:{}\n".format(k,s))
        img = Image.open(k.decode('utf-8'))
        plt.imshow(img)
        pylab.show()


def query_texts_by_image(image_path, topK=3):
    """
    该方法将用于以图搜文。
    需要输入待查询图片的路径和返回的结果数量（topK）。
    该方法会通过CLIP 模型将待查询的图片内容进行预处理，然后通过Vector的TVS.KNNSEARCH命令，查询Tair数据库中与该图片最为吻合的文本。
    将返回目标文本的Key名称和相似距离（distance），其中相似距离（distance）越小，表示相似度越高。
    """
    image_feature = extract_image_features(image_path)
    result = kann_search("index_texts", topK, image_feature)
    print("topK {} q {} result {}",topK,image_feature,result)
    if result == None:
      return None
    for i,item in enumerate(result["vals"]):
        print("{}=>{}\n".format(i,item))



In [29]:
# 宠物图片数据集的路径为“./images”，写入图片数据。
insert_images("./images")

In [38]:
get_index("index_images")

{'name': 'usearch.index_images',
 'dimensions': 1024,
 'metric': 'IP',
 'quantization': 'F64',
 'connectivity': 10,
 'expansion_add': 128,
 'expansion_search': 3,
 'serialization_file_path': '/content/0.usearch.index_images.idx',
 'serialized_length': 61248608,
 'index_size': 7390,
 'index_capacity': 14770,
 'index_mem_usage': 138530384}

In [30]:
# 写入文本示例数据（"狗"、"白色的狗"、"奔跑的白色的狗"）。
insert_text("狗")
insert_text("白色的狗")
insert_text("奔跑的白色的狗")


In [34]:
# 以文搜图，查询最符合文本"奔跑的狗"的三张图。
query_images_by_text("奔跑的狗", 3)

topK {} q {} result {} 3 [-0.005028 -0.01026   0.02936  ... -0.005123  0.03256   0.006367] None


In [36]:
# 以图搜文，指定图片路径，查询比较符合图片描述的文本。
query_texts_by_image("./images/boxer_18.jpg",3)


topK {} q {} result {} 3 [ 0.02061  -0.001164  0.01122  ... -0.00635  -0.03745   0.03198 ] None


In [37]:
!ls ./images/boxer_18.jpg

./images/boxer_18.jpg
